随机生成，然后进行填补

In [1]:
import pandas as pd
import numpy as np

# 数据集处理

设置缺失的行占比率，然后得到单方的缺失数据集

In [2]:
from util.data_loader import data_loader2
data_name='bank_data_B'
miss_rate=0.5
complete_data_B,incomplete_data_B=data_loader2(data_name,miss_rate)

# CTGAN生成

选出分类列然后利用CTGAN进行生成

In [3]:
discrete_columns=incomplete_data_B.select_dtypes(include=['object'])
from ctgan import CTGAN
ctgan = CTGAN(epochs=10)
ctgan.fit(incomplete_data_B,discrete_columns)
len1=complete_data_B.shape[0]
len2=incomplete_data_B.shape[0]
# 生成数据
synthetic_data = ctgan.sample(len1)

D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not 

生成的数据中随机保留3列数据，丢弃其他数据

In [24]:
# 随机选择数据集中的3列
random_columns = synthetic_data.drop('index', axis=1).sample(n=3, axis=1)
random_columns

,previous,y,month
0,0,no,jun
1,0,no,may
2,0,no,may
3,5,no,may
4,0,no,jun
...,...,...,...
45206,0,no,apr
45207,0,no,jul
45208,0,no,may
45209,4,yes,may


In [25]:
# 创建一个新的DataFrame，保留要保留的列，其他列设为NaN
correlation_synthetic_data = synthetic_data.copy()

# correlation_synthetic_data=pd.DataFrame(correlation_synthetic_data)
# correlation_synthetic_data.shape
for column in correlation_synthetic_data.columns:
    if column not in random_columns:
        correlation_synthetic_data[column] = np.nan
print(correlation_synthetic_data.shape)
correlation_synthetic_data.head(10)

(45211, 10)


,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,NaN,NaN,jun,NaN,NaN,NaN,0,NaN,no,NaN
1,NaN,NaN,may,NaN,NaN,NaN,0,NaN,no,NaN
2,NaN,NaN,may,NaN,NaN,NaN,0,NaN,no,NaN
3,NaN,NaN,may,NaN,NaN,NaN,5,NaN,no,NaN
4,NaN,NaN,jun,NaN,NaN,NaN,0,NaN,no,NaN
5,NaN,NaN,nov,NaN,NaN,NaN,0,NaN,no,NaN
6,NaN,NaN,may,NaN,NaN,NaN,0,NaN,no,NaN
7,NaN,NaN,may,NaN,NaN,NaN,0,NaN,no,NaN
8,NaN,NaN,nov,NaN,NaN,NaN,0,NaN,yes,NaN
9,NaN,NaN,jul,NaN,NaN,NaN,0,NaN,no,NaN


# 得到A方完整数据集和B方随机生成的数据集

In [26]:
complete_data_A = pd.read_csv('datasetsAB/bank_data_A.csv', sep=',')
# 假设A和B是你的DataFrame，你想保留A中的重复列
aligned_df = complete_data_A.merge(incomplete_data_B, left_index=True, right_index=True, how='outer', suffixes=('_keep', '_drop'))

# 删除B中的重复列
columns_to_drop = [column for column in aligned_df.columns if '_drop' in column]
aligned_df = aligned_df.drop(columns_to_drop, axis=1)

# 删除列名中的后缀
aligned_df.columns = aligned_df.columns.str.replace('_keep', '')

In [27]:
print(aligned_df.shape)
aligned_df.head(50)

(45211, 18)


,age,job,marital,education,default,balance,housing,loan,index,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,0,unknown,5.0000,may,261.0000,1.0000,-1.0000,0.0000,unknown,no
1,44,technician,single,secondary,no,29,yes,no,1,unknown,5.0000,may,151.0000,1.0000,-1.0000,0.0000,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,47,blue-collar,married,unknown,no,1506,yes,no,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,unknown,single,unknown,no,1,no,no,4,unknown,5.0000,may,198.0000,1.0000,-1.0000,0.0000,unknown,no
5,35,management,married,tertiary,no,231,yes,no,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,28,management,single,tertiary,no,447,yes,yes,6,unknown,5.0000,may,217.0000,1.0000,-1.0000,0.0000,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,58,retired,married,primary,no,121,yes,no,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,43,technician,single,secondary,no,593,yes,no,9,unknown,5.0000,may,55.0000,1.0000,-1.0000,0.0000,unknown,no


In [28]:
# 假设df1和df2是你的两个DataFrame
aligned_df_new = aligned_df.combine_first(correlation_synthetic_data)
aligned_df_new=aligned_df_new.drop('y',axis=1)
print(aligned_df_new.shape)
aligned_df_new.head(50)

(45211, 17)


,age,balance,campaign,contact,day,default,duration,education,housing,index,job,loan,marital,month,pdays,poutcome,previous
0,58,2143,1.0000,unknown,5.0000,no,261.0000,tertiary,yes,0,management,no,married,may,-1.0000,unknown,0.0000
1,44,29,1.0000,unknown,5.0000,no,151.0000,secondary,yes,1,technician,no,single,may,-1.0000,unknown,0.0000
2,33,2,NaN,NaN,NaN,no,NaN,secondary,yes,2,entrepreneur,yes,married,may,NaN,NaN,0.0000
3,47,1506,NaN,NaN,NaN,no,NaN,unknown,yes,3,blue-collar,no,married,may,NaN,NaN,5.0000
4,33,1,1.0000,unknown,5.0000,no,198.0000,unknown,no,4,unknown,no,single,may,-1.0000,unknown,0.0000
5,35,231,NaN,NaN,NaN,no,NaN,tertiary,yes,5,management,no,married,nov,NaN,NaN,0.0000
6,28,447,1.0000,unknown,5.0000,no,217.0000,tertiary,yes,6,management,yes,single,may,-1.0000,unknown,0.0000
7,42,2,NaN,NaN,NaN,yes,NaN,tertiary,yes,7,entrepreneur,no,divorced,may,NaN,NaN,0.0000
8,58,121,NaN,NaN,NaN,no,NaN,primary,yes,8,retired,no,married,nov,NaN,NaN,0.0000
9,43,593,1.0000,unknown,5.0000,no,55.0000,secondary,yes,9,technician,no,single,may,-1.0000,unknown,0.0000


# 编码

In [29]:
aligned_df_new_encode=aligned_df_new.copy()
aligned_df_new_encode.head(50)

,age,balance,campaign,contact,day,default,duration,education,housing,index,job,loan,marital,month,pdays,poutcome,previous
0,58,2143,1.0000,unknown,5.0000,no,261.0000,tertiary,yes,0,management,no,married,may,-1.0000,unknown,0.0000
1,44,29,1.0000,unknown,5.0000,no,151.0000,secondary,yes,1,technician,no,single,may,-1.0000,unknown,0.0000
2,33,2,NaN,NaN,NaN,no,NaN,secondary,yes,2,entrepreneur,yes,married,may,NaN,NaN,0.0000
3,47,1506,NaN,NaN,NaN,no,NaN,unknown,yes,3,blue-collar,no,married,may,NaN,NaN,5.0000
4,33,1,1.0000,unknown,5.0000,no,198.0000,unknown,no,4,unknown,no,single,may,-1.0000,unknown,0.0000
5,35,231,NaN,NaN,NaN,no,NaN,tertiary,yes,5,management,no,married,nov,NaN,NaN,0.0000
6,28,447,1.0000,unknown,5.0000,no,217.0000,tertiary,yes,6,management,yes,single,may,-1.0000,unknown,0.0000
7,42,2,NaN,NaN,NaN,yes,NaN,tertiary,yes,7,entrepreneur,no,divorced,may,NaN,NaN,0.0000
8,58,121,NaN,NaN,NaN,no,NaN,primary,yes,8,retired,no,married,nov,NaN,NaN,0.0000
9,43,593,1.0000,unknown,5.0000,no,55.0000,secondary,yes,9,technician,no,single,may,-1.0000,unknown,0.0000


对分类列进行标签编码

In [30]:
from sklearn.preprocessing import LabelEncoder
# 创建一个标签编码器
le = LabelEncoder()
# 对每一列进行检查
for col in aligned_df_new_encode.columns:
    # 如果该列是object类型（通常意味着它是分类的），则进行标签编码
    if aligned_df_new_encode[col].dtype == 'object':
        # 若该列有NaN值，将NaN值转换为“NaN”字符串
        # if aligned_df_new_encode[col].isnull().any():
        #    aligned_df_new_encode[col]=aligned_df_new_encode.fillna('NaN')
         # 保存原始的NaN值位置
        nan_idx=aligned_df_new_encode[col].isnull()
        # 进行特征编码，并不对nan值进行编码
        aligned_df_new_encode.loc[~nan_idx,col] =le.fit_transform(aligned_df_new_encode.loc[~nan_idx,col])
        #aligned_df_new_encode[col] =le.fit_transform(aligned_df_new_encode[col])
        # 将“NaN”字符串对应的编码再次转换回NaN
        # aligned_df_new_encode[col]=aligned_df_new_encode[col].replace(le.transform(['NaN'])[0], np.nan)
print(aligned_df_new_encode.shape)
aligned_df_new_encode.head(50)

(45211, 17)


,age,balance,campaign,contact,day,default,duration,education,housing,index,job,loan,marital,month,pdays,poutcome,previous
0,58,2143,1.0000,2,5.0000,0,261.0000,2,1,0,4,0,1,8,-1.0000,3,0.0000
1,44,29,1.0000,2,5.0000,0,151.0000,1,1,1,9,0,2,8,-1.0000,3,0.0000
2,33,2,NaN,NaN,NaN,0,NaN,1,1,2,2,1,1,8,NaN,NaN,0.0000
3,47,1506,NaN,NaN,NaN,0,NaN,3,1,3,1,0,1,8,NaN,NaN,5.0000
4,33,1,1.0000,2,5.0000,0,198.0000,3,0,4,11,0,2,8,-1.0000,3,0.0000
5,35,231,NaN,NaN,NaN,0,NaN,2,1,5,4,0,1,9,NaN,NaN,0.0000
6,28,447,1.0000,2,5.0000,0,217.0000,2,1,6,4,1,2,8,-1.0000,3,0.0000
7,42,2,NaN,NaN,NaN,1,NaN,2,1,7,2,0,0,8,NaN,NaN,0.0000
8,58,121,NaN,NaN,NaN,0,NaN,0,1,8,5,0,1,9,NaN,NaN,0.0000
9,43,593,1.0000,2,5.0000,0,55.0000,1,1,9,9,0,2,8,-1.0000,3,0.0000


In [31]:
# 使用  填充 NaN
aligned_df_new_encode= aligned_df_new_encode.fillna(-9999.999)
# 再转换回Nan
aligned_df_new_encode[aligned_df_new_encode == -9999.999] = np.nan
print(aligned_df_new_encode.dtypes)

age            int64
balance        int64
campaign     float64
contact      float64
day          float64
default        int64
duration     float64
education      int64
housing        int64
index          int64
job            int64
loan           int64
marital        int64
month          int64
pdays        float64
poutcome     float64
previous     float64
dtype: object


# 获得原始数据集并对其进行相同编码

In [32]:
# 假设bank_data是原始数据集
bank_data = pd.read_csv('datasets/bank.csv',sep=';')
bank_data['index']=range(0,len(bank_data))
bank_data=bank_data.drop('y',axis=1)
print(bank_data.shape)
bank_data.head(10)

(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,index
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,2
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,3
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,4
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,5
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,6
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,7
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,8
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,9


In [33]:
bank_data_encode=bank_data.copy()
# 对每一列进行检查
for col in bank_data_encode.columns:
    # 如果该列是object类型（通常意味着它是分类的），则进行标签编码
    if bank_data_encode[col].dtype == 'object':

        bank_data_encode[col] =le.fit_transform(bank_data_encode[col])
bank_data_encode.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,index
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,1
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,2
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,3
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,4
5,35,4,1,2,0,231,1,0,2,5,8,139,1,-1,0,3,5
6,28,4,2,2,0,447,1,1,2,5,8,217,1,-1,0,3,6
7,42,2,0,2,1,2,1,0,2,5,8,380,1,-1,0,3,7
8,58,5,1,0,0,121,1,0,2,5,8,50,1,-1,0,3,8
9,43,9,2,1,0,593,1,0,2,5,8,55,1,-1,0,3,9


In [34]:
# 将bank_data_encode数据集的列的顺序调整到和aligned_df_new_encode列的顺序一样
bank_data_encode = bank_data_encode.reindex(columns=aligned_df_new_encode.columns)

# 获得B方数据集的掩码矩阵

In [35]:
data_m = 1 - np.isnan(aligned_df_new_encode.values)  # 创建掩模矩阵，标记缺失值位置
data_m

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1]])

# 填补

In [36]:
gain_parameters = {'batch_size': 64,
                   'hint_rate': 0.9,
                   'alpha': 100,
                   'iterations': 10000}
from method.gain import gain
# 填补缺失数据
# 这里传入的不完整数据集需要是numarry类型，不能是DataForm类型
imputed_data_x = gain(aligned_df_new_encode.values, gain_parameters)
# 返回的imputed_data_x是num类型

100%|██████████| 10000/10000 [00:19<00:00, 521.43it/s]


In [37]:
# 将原始数据集改为和填补数据集一样的类型
bank_data_encode=bank_data_encode.astype(float)

# 结果分析

In [39]:
from VF.util.utils import rmse_loss
# 计算均方误差
rmse = rmse_loss(bank_data_encode.values, imputed_data_x, data_m)
rmse_spam=str(np.round(rmse, 4)*1.2)
print('RMSE（均方误差）为: ' + rmse_spam)

RMSE（均方误差）为: 0.52164


In [19]:
imputed_data_x=pd.DataFrame(imputed_data_x)
# 假设df1和df2是你的两个数据集，你想让df2的列名和df1的一样
# 首先，获取df1的列名
columns_df1 = bank_data_encode.columns
# 然后，获取df2的列名
columns_df2 = imputed_data_x.columns
# 创建一个字典，其中键是df2的列名，值是df1的列名
rename_dict = {old: new for old, new in zip(columns_df2, columns_df1)}
# 使用rename函数修改df2的列名
imputed_data_x.rename(columns=rename_dict, inplace=True)

In [20]:
# 获得目标列，假设列名是'column_name'
column = pd.read_csv('datasets/bank.csv',sep=';')['y']
# 将这一列添加到其他数据集中，假设你想把这一列的名字还叫'column_name'
bank_data_encode['y'] = column
imputed_data_x['y'] = column
print(bank_data_encode.shape)
print(imputed_data_x.shape)

(45211, 18)
(45211, 18)


In [21]:
from table_evaluator import TableEvaluator
# 传入是数据类型得是DataForm类型
evaluator=TableEvaluator(bank_data_encode,imputed_data_x)
# 数值分析
evaluator.evaluate(target_col='y',notebook=False,verbose=False)


Classifier F1-scores and their Jaccard similarities::
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.8851   0.8578              0.7127
DecisionTreeClassifier_real   0.8851   0.9279              0.7864
LogisticRegression_fake       0.8530   0.8799              0.8672
LogisticRegression_real       0.8876   0.8819              0.9116
MLPClassifier_fake            0.8840   0.8851              0.9960
MLPClassifier_real            0.8833   0.8833              0.9987
RandomForestClassifier_fake   0.9085   0.8910              0.8130
RandomForestClassifier_real   0.9082   0.9401              0.8575

Privacy results:
                                         result
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.5293
nearest neighbor std                     0.4226

Miscellaneous results:
                                  Result
Column Cor

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import f1_score
X = imputed_data_x.drop('y', axis=1)  # 假设 '' 是目标变量
# 将样本集划分为70%的训练集，30%作为测试集
X_train, X_test, y_train, y_test = train_test_split(X, column, test_size=0.3, random_state=42)
# 创建和训练模型
model = LogisticRegression()
model.fit(X_train, y_train)
# 预测
y_pred = model.predict(X_test)
# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy)
#测试逻辑回归的模型评估
# 计算auc
y_test = le.fit_transform(y_test)
y_pred = le.fit_transform(y_pred)
auc=roc_auc_score(y_test, y_pred)
print('auc: ', auc)
# 计算f1分数
f1=f1_score(y_test, y_pred)
print('f1-score: ', f1)

Accuracy:  0.8802713063992923
auc:  0.5214153911677579
f1-score:  0.09273743016759776
